In [44]:
import pandas as pd
from sklearn import linear_model,metrics
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from scipy.stats import boxcox
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
from datetime import date
sns.set()
%matplotlib inline

In [45]:
train_df = pd.read_csv("csv/06204_train_prepared.csv")
test_df = pd.read_csv("csv/06204_test_prepared.csv")

In [46]:
test_id = test_df["PassengerId"]

### Logisticやります
やりかたよくわかってない
  
- `train loop`を決める(?)

In [58]:
def train(df):
    df["preds"] = pd.NA

    drop = ["Transported","preds","kfold"]

    for fold in range(5):
        train = df[df["kfold"] != fold]

        y_train = train["Transported"].values
        x_train = train.drop(drop,axis=1).values

        val = df[df["kfold"] == fold]

        y_val = val["Transported"].values
        x_val = val.drop(drop,axis=1).values

        # model = RandomForestClassifier(max_depth=10,max_features="auto",n_estimators=100)
        # model = linear_model.LogisticRegression(max_iter=1000)
        model = XGBClassifier(colsample_bytree=1.0,eval_metric="mlogloss",learning_rate=0.1,min_child_weight=2,max_depth=5,n_estimators=50,subsample=1.0)
        model.fit(x_train,y_train)

        preds = model.predict(x_val)
        df.loc[val.index,"preds"] = preds

        acc = metrics.accuracy_score(y_val,preds)
        print(f"Fold{fold+1}-Accuracy = {acc:.4f}")
    
    df[drop] = df[drop].astype(int)

    # 全体の正解率計算
    acc = metrics.accuracy_score(df["Transported"].values,df["preds"].values)
    print(f"Overall accuracy:{acc:.4f}")

    return df,model

In [48]:
test_id = test_df["PassengerId"]
del test_df["PassengerId"]

In [49]:
test_df_values = test_df.values


In [50]:
train_df.head()

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Alone,Group_zero,RoomService_missing,FoodCourt_missing,ShoppingMall_missing,Cabin_missing,VIP_missing,TotalExpense_missing,TotalExpense,CabinSide,No_Spending,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,GroupSize_1,GroupSize_2,GroupSize_3,GroupSize_4,GroupSize_5,GroupSize_6,GroupSize_7,GroupSize_8,CabinDeck_A,CabinDeck_B,CabinDeck_C,CabinDeck_D,CabinDeck_E,CabinDeck_F,CabinDeck_G,CabinDeck_T,Transported,kfold
0,0.0,0.702095,-0.337025,-0.284274,-0.287317,-0.273736,-0.266098,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.514066,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,-0.333233,-0.173528,-0.278689,-0.245971,0.209267,-0.227692,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.251479,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3
2,0.0,2.013510,-0.272527,1.934922,-0.287317,5.634034,-0.223327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.190333,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.287964,-0.337025,0.511931,0.326250,2.655075,-0.097634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.332604,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,0.0,-0.885407,0.117466,-0.240833,-0.037590,0.223344,-0.264352,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.124824,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,4


In [59]:
# CabinNumとGrpupId以外
train_df_exp1,model = train(train_df.copy())

Fold1-Accuracy = 0.7890
Fold2-Accuracy = 0.7987
Fold3-Accuracy = 0.8131
Fold4-Accuracy = 0.8055
Fold5-Accuracy = 0.8090
Overall accuracy:0.8031


In [52]:
pred = model.predict(test_df_values)

In [53]:
ans_df = pd.DataFrame()
ans_df["PassengerId"] = test_id
ans_df["Transported"] = pred.astype(bool)

In [54]:
def submit_date(count=1):
    d = "submit/submit_"
    today = date.today()
    d += f"{today.month}{today.day}_{count}.csv"

    return d

In [55]:
ans_df.to_csv(submit_date(2),index=False)

### `train`関数
1. `Transported`,`preds`,`kfold`は学習に使わないよ！

``` python
for fold in range(5)
```

2. kfoldの回数である5回回して学習させるよ！

``` python
train = df[df["kfold"] != fold]
```

- 各イテレートにおうじてtrainデータ決める
- `y_train`は到着できたかどうか
- `x_train`は各データ
- `y_val`,`x_val`も一緒

3. modelはロジスティック回帰を`max_iter`1000にするよ！
4. 学習！
5. 予測！
6. dfに代入！！
7. 正解率計算！
8. owari

In [56]:
#1.0が正解，0.0がミスとして見てみたい
wrong = train_df_exp1[train_df_exp1["Transported"] != train_df_exp1["preds"]]

In [57]:
wrong["GroupSize_1"].value_counts()

1.0    1044
0.0     668
Name: GroupSize_1, dtype: int64